In [1]:
import pandas as pd
import dtale
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [2]:
df = pd.read_csv('data_original.csv')
df.drop(columns=['make','price_position'], inplace=True)
df.replace('Unlisted', np.nan, inplace=True)

## Create test set
- Will use CV for model selection
- Test set to evaluate model at the end
- Will perform all processing on test set simultaneously with train (avoiding leakage)

In [3]:
df, test_df = train_test_split(df, test_size=0.1, random_state=123)

## Create features from 'derivative' column

In [4]:
def derivative_features(df_):
    df_['euro_emissions'] = df_['derivative'].str.extract(r'Euro (\d)').astype('float') ## The emissions classification (ULEZ etc)
    df_['stopstart'] = df_['derivative'].str.contains('(s/s)', na=False)
    df_['motorsport'] = df_['derivative'].str.contains('M Sport', na=False)
    df_['sport'] = df_['derivative'].str.contains('Sport', na=False) & ~df['derivative'].str.contains('M Sport', na=False)
    df_['doors'] = df_['derivative'].str.extract(r'(\d)dr').astype('float')
    df_['d_number'] = df_['derivative'].str.extract(r'(\d{2,})d ').astype('float') 
    df_['i_number'] = df_['derivative'].str.extract(r'(\d{2,})i ').astype('float') 
    df_['e_number'] = df_['derivative'].str.extract(r'(\d{2,})e ').astype('float') 
    df_['xdrive'] = df_['derivative'].str.contains('xDrive', na=False)
    df_['se'] = df_['derivative'].str.contains(' SE ', na=False)
    df_['steptronic'] = df_['derivative'].str.contains('Steptronic', na=False)
    df_['dct'] = df_['derivative'].str.contains('DCT', na=False) # dual clutch transmission
    df_['touring'] = df_['derivative'].str.contains('Touring', na=False) 
    df_['special_edition'] = df_['derivative'].str.contains('Edition', na=False) 
    df_['competition'] = df_['derivative'].str.contains('Competition', na=False) 
    df_['battery_capacity'] = df_['derivative'].str.extract(r'(\d+\.\d+|\d+)kWh').astype('float')
    return df_

df = derivative_features(df)
test_df = derivative_features(test_df)

/var/folders/qt/w00br2yn2x5fzq44vmf5n86h0000gn/T/ipykernel_22992/3882310352.py:3: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.

/var/folders/qt/w00br2yn2x5fzq44vmf5n86h0000gn/T/ipykernel_22992/3882310352.py:3: UserWarning:

This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.



### Fill null fuel type
- Deduce fuel_type from the d_number, i_number and e_number/engine_size
- Otherwise, use the mode for that car model, then the mode for the whole dataset

In [5]:
print('Number of cars with null fuel_type before:', len(df[df['fuel_type'].isnull()]))

def fill_fuel_type_by_deduction(row):
    if pd.isnull(row['fuel_type']):
        if not pd.isnull(row['d_number']):
            return 'Diesel'
        if not pd.isnull(row['i_number']):
            return 'Petrol'
        if not pd.isnull(row['e_number']):
            if pd.isnull(row['engine_size']) or row['engine_size']==0:
                return 'Electric'
            else:
                return 'Petrol Plug-in Hybrid'
    return row['fuel_type']

df['fuel_type'] = df.apply(fill_fuel_type_by_deduction, axis=1)
test_df['fuel_type'] = test_df.apply(fill_fuel_type_by_deduction, axis=1)

# Fill NaN values in 'fuel_type' using the mode for each model
modes = df.groupby('model')['fuel_type'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df['fuel_type'] = df.apply(
    lambda row: modes[row['model']] if pd.isnull(row['fuel_type']) else row['fuel_type'], 
    axis=1
)

test_df['fuel_type'] = test_df.apply(
    lambda row: modes[row['model']] if ((pd.isnull(row['fuel_type'])) & (row['model'] in modes)) else row['fuel_type'], 
    axis=1
)

# Failing that, fill with mode for whole dataset
df['fuel_type'] = df['fuel_type'].fillna(df['fuel_type'].mode()[0])
test_df['fuel_type'] = test_df['fuel_type'].fillna(df['fuel_type'].mode()[0])

print('Number of cars with null fuel_type after:', len(df[df['fuel_type'].isnull()]))

Number of cars with null fuel_type before: 4
Number of cars with null fuel_type after: 0


## Electric cars: fix contradictions between fuel_type=='Electric' and engine_size
- Some cars have fuel_type=='Electric' and engine size can be 0,1 or null - want this to be uniform
- Some cars have engine_size==0 and fuel_type!='Electric' - set this as null so it can be corrected later

In [6]:
print('Number of electric cars with null engine_size:', len(df[(df['fuel_type']=='Electric') & (df['engine_size'].isna())]))
df[df['fuel_type']=='Electric'].groupby('engine_size').count()

Number of electric cars with null engine_size: 102


,model,body_type,fuel_type,transmission,derivative,plate,year,mileage,price,colour,area,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,euro_emissions,stopstart,motorsport,sport,doors,d_number,i_number,e_number,xdrive,se,steptronic,dct,touring,special_edition,competition,battery_capacity
engine_size,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,285,285,285,285,285,281,281,285,285,285,285,285,285,285,285,285,285,285,285,285,285,0,285,285,285,93,0,0,0,285,285,285,285,285,285,285,84
1.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1


In [7]:
def electric_engine_size(df_):
    df_.loc[(df_['engine_size']==0) & (df_['fuel_type']!='Electric'), 'engine_size'] = np.nan
    df_.loc[(df_['engine_size']!=0) & (df_['fuel_type']=='Electric'), 'engine_size'] = 0
    df_.loc[(df_['engine_size'].isna()) & (df_['fuel_type']=='Electric'), 'engine_size'] = 0
    return df_

df = electric_engine_size(df)
test_df = electric_engine_size(test_df)

## Fill null engine size 
- Use mode for that model and fuel type
- Otherwise use mode for fuel type

In [8]:
print('Number of cars with null engine_size before:', df['engine_size'].isna().sum())

modes = df.groupby(['model','fuel_type'])['engine_size'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df['engine_size'] = df.apply(
    lambda row: modes[row['model'], row['fuel_type']] if pd.isnull(row['engine_size']) else row['engine_size'], 
    axis=1
)
test_df['engine_size'] = test_df.apply(
    lambda row: modes[row['model'], row['fuel_type']] if ((pd.isnull(row['engine_size'])) & ((row['model'], row['fuel_type']) in modes)) else row['engine_size'], 
    axis=1
)

modes = df.groupby(['fuel_type'])['engine_size'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df['engine_size'] = df.apply(
    lambda row: modes[row['fuel_type']] if pd.isnull(row['engine_size']) else row['engine_size'], 
    axis=1
)
test_df['engine_size'] = test_df.apply(
    lambda row: modes[row['fuel_type']] if pd.isnull(row['engine_size']) else row['engine_size'], 
    axis=1
)

print('Number of cars with null engine_size after:', df['engine_size'].isna().sum())

Number of cars with null engine_size before: 111
Number of cars with null engine_size after: 0


## Set engine size to null for electrics

In [9]:
df.loc[(df['fuel_type']=='Electric'), 'engine_size'] = np.nan
test_df.loc[(test_df['fuel_type']=='Electric'), 'engine_size'] = np.nan

## Are there any cars where the plate contradicts the year?
- There are only two cars where plate and year don't match
- We don't know whether the plate or the year is correct -> set both to null and deal with them later

In [10]:
plates_dict = {year : [str(year-2000), str(year-1950), str(year-1951)] for year in range(2002, 2024)}
accepted_numerical_plates = pd.DataFrame([(year, plate) for year, plates in plates_dict.items() for plate in plates], columns=['year', 'plate'])
accepted_numerical_plates['plate']=accepted_numerical_plates['plate'].astype(str)

accepted_numerical_plates = pd.DataFrame([(year, plates) for year, plates in plates_dict.items()], 
                                         columns=['year', 'accepted_plates'])

def check_accepted_plates(df_, accepted_numerical_plates):
    df_ = df_.merge(accepted_numerical_plates, how='left', on='year')
    df_.loc[pd.isna(df_['accepted_plates']), 'accepted_plates'] = df_['plate'][pd.isna(df_['accepted_plates'])].apply(lambda x: [x])
    
    df_['is_accepted'] = df_.apply(lambda row: True if pd.isna(row['plate']) else row['plate'] in row['accepted_plates'], axis=1)
    print(len(df_[~df_['is_accepted']]), 'cars do not have acceptable plates')
    df_.loc[(~df_['is_accepted']), 'year'] = np.nan
    df_.loc[(~df_['is_accepted']), 'plate'] = np.nan
    df_ = df_.drop(columns=['accepted_plates', 'is_accepted'])
    return df_

df = check_accepted_plates(df, accepted_numerical_plates)
test_df = check_accepted_plates(test_df, accepted_numerical_plates)

2 cars do not have acceptable plates
0 cars do not have acceptable plates


## Group all older plates
- Found that the older plates don't impact the model so can group them together

In [11]:
df.loc[df['plate'].str.isalpha().fillna(False), 'plate'] = 'alpha'
test_df.loc[test_df['plate'].str.isalpha().fillna(False), 'plate'] = 'alpha'

## Fill missing plate / year
- For missing plate, fill with the mode plate for cars of that year, and vice versa

In [12]:
modes = df.groupby(['year'])['plate'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df['plate'] = df.apply(
    lambda row: (modes[row['year']] if row['year'] in list(modes.index) else None) if pd.isnull(row['plate']) else row['plate'], 
    axis=1
)
test_df['plate'] = test_df.apply(
    lambda row: (modes[row['year']] if row['year'] in list(modes.index) else None) if pd.isnull(row['plate']) else row['plate'], 
    axis=1
)

modes = df.groupby(['plate'])['year'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
df['year'] = df.apply(
    lambda row: (modes[row['plate']] if row['plate'] in list(modes.index) else None) if pd.isnull(row['year']) else row['year'], 
    axis=1
)
test_df['year'] = test_df.apply(
    lambda row: (modes[row['plate']] if row['plate'] in list(modes.index) else None) if pd.isnull(row['year']) else row['year'], 
    axis=1
)

## Use 'car age' instead of 'year of manufacture'

In [13]:
df['age'] = 2023 - df['year']
df = df.drop(columns=['year'])

test_df['age'] = 2023 - test_df['year']
test_df = test_df.drop(columns=['year'])

## Now use a regression model to impute the age
- Age is an important feature for the model
- Can impute it by using the fact that the mileage and age have a close to linear relationship
- Force intercept to zero

In [14]:
train_data = df[['mileage','age']].dropna()
X_train = train_data[['mileage']]
y_train = train_data['age']

model = LinearRegression(fit_intercept=False).fit(X_train, y_train)

df.loc[df['age'].isnull(), 'age'] = model.predict(df[df['age'].isnull()][['mileage']])
df['age'] = df['age'].astype(int)
test_df.loc[test_df['age'].isnull(), 'age'] = model.predict(test_df[test_df['age'].isnull()][['mileage']])
test_df['age'] = test_df['age'].astype(int)

In [15]:
coefficients = model.coef_
print("Coefficients:", coefficients)
intercept = model.intercept_
print("Intercept:", intercept)

Coefficients: [0.0001115]
Intercept: 0.0


- Now fill the plate if missing

In [16]:
modes = df.groupby(['age'])['plate'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)

df['plate'] = df.apply(
    lambda row: (modes[row['age']] if row['age'] in list(modes.index) else None) if pd.isnull(row['plate']) else row['plate'], 
    axis=1
)
test_df['plate'] = test_df.apply(
    lambda row: (modes[row['age']] if row['age'] in list(modes.index) else None) if pd.isnull(row['plate']) else row['plate'], 
    axis=1
)

## New plate feature - first, middle or end part of year
- Then can drop plate

In [17]:
def new_plate_features(df_):
    df_['plate_yearstart'] = 72 - df_['age'] == pd.to_numeric(df_['plate'], errors='coerce').fillna(False)
    df_['plate_yearmiddle'] = 23 - df_['age'] == pd.to_numeric(df_['plate'], errors='coerce').fillna(False)
    df_['plate_yearend'] = 73 - df_['age'] == pd.to_numeric(df_['plate'], errors='coerce').fillna(False)
    
    df_['plate_time_of_year'] = df_.apply(lambda row: 1 if row['plate_yearstart'] else 2 if row['plate_yearmiddle'] else 3 if row['plate_yearend'] else np.nan, 1)
    df_.drop(columns=['plate'], inplace=True)
    return df_

df = new_plate_features(df)
test_df = new_plate_features(test_df)

## Rare models -> 'Other'
- Lots of models only have one car
- Create 'Other' car model to replace these with

In [18]:
df.loc[df['model'].str.contains('Alpina', case=False, na=False), 'model'] = 'Alpina'
test_df.loc[test_df['model'].str.contains('Alpina', case=False, na=False), 'model'] = 'Alpina'

rare_models = df.groupby('model')['price'].count().reset_index()
rare_models = rare_models[rare_models['price']<10]

df.loc[df['model'].isin(list(rare_models['model'])), 'model'] = 'Other'
test_df.loc[test_df['model'].isin(list(rare_models['model'])), 'model'] = 'Other'

## Fill nulls in body type and transmission
- Check the 'derivative' column to see if it contains the body type. Otherwise use the mode for the dataset.

In [19]:
body_types = df['body_type'].dropna().unique()

def fill_body_type(row):
    if pd.isnull(row['body_type']):
        if pd.isnull(row['derivative']):
            return row['body_type']
        for type in body_types:
            if type in row['derivative']:
                return type
    return row['body_type']

df['body_type'] = df.apply(fill_body_type, axis=1)
test_df['body_type'] = test_df.apply(fill_body_type, axis=1)

df['body_type'].fillna(df['body_type'].mode()[0], inplace=True)
test_df['body_type'].fillna(df['body_type'].mode()[0], inplace=True)
df['transmission'].fillna(df['transmission'].mode()[0], inplace=True)
test_df['transmission'].fillna(df['transmission'].mode()[0], inplace=True)

## Reduce number of colour features
- Some were not important in model, some only have a small number of cars

In [20]:
colours_to_keep = ['Orange', 'Green', 'White', 'Silver', 'Black', 'Blue', 'Grey', 'Red', 'Yellow']
df['colour'] = df['colour'].replace([col for col in df['colour'].unique() if col not in colours_to_keep], 'Other')
test_df['colour'] = test_df['colour'].replace([col for col in test_df['colour'].unique() if col not in colours_to_keep], 'Other')

## Create a 'mileage per year' feature

In [21]:
df['mileage_per_year'] = df['mileage']/(df['age']+1)
test_df['mileage_per_year'] = test_df['mileage']/(test_df['age']+1)

## Drop columns:
- Drop derivative
- Drop area - it doesn't add anything to the model in its current form and no time to do any more analysis

In [22]:
df = df.drop(columns=['area', 'derivative'])
test_df = test_df.drop(columns=['area', 'derivative'])

In [23]:
df.to_csv('processed_data.csv', index=False)

In [24]:
test_df.to_csv('processed_test_data.csv', index=False)